# Major Project - LSDL Model

In [ ]:
import torch
import random
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.decomposition import TruncatedSVD
from matplotlib.backends.backend_pdf import PdfPages  
from sklearn.preprocessing import MaxAbsScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.linear_model import LinearRegression

## Data Preparation

### Data Configuration:

In [ ]:
DATASET_CONFIGS = {
    "Traffic": {
        "filename": "PeMS", 
        "P": 24, "W": 48, "CK": 6, "horizons": [3, 6, 12, 24], "resample": False
    },
    "Solar": {
        "filename": "solar_Alabama", 
        "P": 144, "W": 36, "CK": 6, "horizons": [3, 6, 12, 24], "resample": False
    },
    "Electricity": {
        "filename": "LD2011_2014", 
        "P": 24, "W": 48, "CK": 6, "horizons": [3, 6, 12, 24], "resample": "1H"
    },
    "Exchange": {
        "filename": "exchange", 
        "P": 7, "W": 14, "CK": 6, "horizons": [3, 6, 12, 24], "resample": False
    }
}

### Load and Process Data:

In [ ]:
def load_and_process_data(config_name):
    cfg = DATASET_CONFIGS[config_name]
    fname = cfg["filename"]
    print(f"\n[Loader] Reading {config_name} from '{fname}'...")
    try:
        if config_name == "Exchange":
            df = pd.read_csv(f"{fname}.csv", sep=',', decimal='.', header=0)
        else:
            try:
                df = pd.read_csv(f"datset\{fname}", header=0)
            except FileNotFoundError:
                df = pd.read_csv(f"{fname}.csv", header=0)
                if config_name == 'exchange': print(df.head())

        df_numeric = df.select_dtypes(include=[np.number])
        
        if cfg["resample"] == "1H":
            print("   -> Resampling to Hourly...")
            data_values = df_numeric.values
            limit = (data_values.shape[0] // 4) * 4
            data_values = data_values[:limit].reshape(-1, 4, data_values.shape[1]).mean(axis=1)
            data_values = data_values[-26304:]
            PAPER_CLIENT_COUNT = 321
            if data_values.shape[1] > PAPER_CLIENT_COUNT:
                print(f"   -> Filtering Clients: Keeping top {PAPER_CLIENT_COUNT} active users...")
                client_activity = np.mean(data_values, axis=0)            
                top_client_indices = np.argsort(client_activity)[-PAPER_CLIENT_COUNT:]
                top_client_indices = np.sort(top_client_indices)
                data_values = data_values[:, top_client_indices] 
                df_numeric = pd.DataFrame(data_values)

        return df_numeric.values
    
    except Exception as e:
        print(f"   [Error] Could not load {fname}: {e}")
        return None

### Creating the dataset

In [ ]:
def create_dataset(data, window_size, period, horizon):

    # Constructs X (Long-term) and Q (Short-term) inputs.

    X_long, Q_short, Y = [], [], []
    
    # We need enough history for the long-term component.
    # Assuming long-term looks back at 'period' intervals.
    start_index = max(window_size, period * window_size) 
    
    for i in range(start_index, len(data) - horizon):
        # Short-term (Q): Immediate history [i-W : i]
        short_term = data[i - window_size : i]
        
        # Long-term (X): Historical periodic data
        # We sample the same time of day from previous days
        indices = [i - (p * period) for p in range(window_size, 0, -1)]
        long_term = data[indices]
        
        # Target (Y): The value at horizon
        target = data[i + horizon - 1]
        
        X_long.append(long_term)
        Q_short.append(short_term)
        Y.append(target)
        
    return np.array(X_long), np.array(Q_short), np.array(Y)

## LS-DL Model Architecture

### General Configuration


In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

### Model Architecture Definition:

### 1) LS-DL Model Architecture:

In [ ]:
class LSDL(nn.Module):
    def __init__(self, num_variables, window_size, horizon=1):
        super(LSDL, self).__init__()
        
        # --- Hyperparameters (Table 4) ---
        self.conv_hidden = 200
        self.rnn_hidden = 100
        self.dropout_rate = 0.0
        self.kernel_size = 6
        self.ar_window = 12 
        
        # --- Short-term Branch (CNN + GRU) ---
        self.cnn_short = nn.Conv1d(num_variables, self.conv_hidden, self.kernel_size)
        self.relu_short = nn.ReLU()
        self.gru_short = nn.GRU(self.conv_hidden, self.rnn_hidden, batch_first=True)
        self.dropout_short = nn.Dropout(self.dropout_rate)

        # --- Long-term Branch (CNN + GRU) ---
        self.cnn_long = nn.Conv1d(num_variables, self.conv_hidden, self.kernel_size)
        self.relu_long = nn.ReLU()
        self.gru_long = nn.GRU(self.conv_hidden, self.rnn_hidden, batch_first=True)
        self.dropout_long = nn.Dropout(self.dropout_rate)

        # --- Fully Connected Combination ---
        self.fc_combine = nn.Linear(self.rnn_hidden * 2, num_variables)
        
        # --- FIXED AUTOREGRESSIVE COMPONENT (Equation 7) ---
        # We use Conv1d with groups=num_variables to ensure
        # Sensor A only predicts Sensor A (Component-wise AR).
        # This drastically reduces parameters and prevents "explosion".
        self.ar = nn.Conv1d(
            in_channels=num_variables, 
            out_channels=num_variables, 
            kernel_size=self.ar_window, 
            groups=num_variables  # Key Fix: Independent weights per sensor
        )

        self._init_ar_weights()

    def _init_ar_weights(self):
        """
        Manually sets the AR weights so the model starts by predicting:
        y_t = 1.0 * y_{t-1} + 0.0 * others
        This forces the starting RRSE to be reasonable (~0.6) instead of random (>1.0).
        """
        # 1. Zero out all AR weights and bias
        nn.init.constant_(self.ar.weight, 0)
        nn.init.constant_(self.ar.bias, 0)
        
        # 2. Set the "most recent" lag to 1.0 for every sensor
        # The kernel shape is (Out, In/Groups, Kernel).
        # We want the last index of the kernel (most recent time) to be 1.
        with torch.no_grad():
            for i in range(self.ar.out_channels):
                # Set the last weight (most recent history) to 1.0
                self.ar.weight[i, 0, -1] = 0.6

    def forward(self, x_long, x_short):
        # 1. Short-term Branch
        s_in = x_short.permute(0, 2, 1) # (Batch, Vars, Time)
        s_c = self.relu_short(self.cnn_short(s_in)).permute(0, 2, 1)
        _, s_h = self.gru_short(s_c)
        s_h = self.dropout_short(s_h.squeeze(0))

        # 2. Long-term Branch
        l_in = x_long.permute(0, 2, 1)
        l_c = self.relu_long(self.cnn_long(l_in)).permute(0, 2, 1)
        _, l_h = self.gru_long(l_c)
        l_h = self.dropout_long(l_h.squeeze(0))

        # 3. Deep Network Output (Equation 6)
        combined = torch.cat((l_h, s_h), dim=1)
        nn_pred = self.fc_combine(combined) # (Batch, Vars)
        
        # 4. FIXED AR Calculation (Equation 7)
        # We only take the last 'ar_window' (12) steps from the short-term history
        ar_input = x_short.permute(0, 2, 1)[:, :, -self.ar_window:] 
        ar_pred = self.ar(ar_input) # Output: (Batch, Vars, 1)
        ar_pred = ar_pred.squeeze(-1) # Output: (Batch, Vars)

        # 5. Final Sum (Equation 8)
        return nn_pred + ar_pred

### 2) Baseline GRU Model Architecture:

In [ ]:
class BaselineGRU(nn.Module):
    def __init__(self, num_variables, hidden_size=100):
        super(BaselineGRU, self).__init__()
        self.gru = nn.GRU(input_size=num_variables, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_variables)
    
    def forward(self, x_long, x_short):
        _, h_n = self.gru(x_short)
        return self.fc(h_n.squeeze(0))

### 3) VARMLP (Vector AR + MLP):

In [ ]:
class VARMLP(nn.Module):
    def __init__(self, num_variables, window_size, hidden_dim=100):
        super(VARMLP, self).__init__()
        # MLP Component: Flattens input -> Dense -> Output
        self.mlp = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_variables * window_size, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_variables)
        )
        # AR Component: Linear transformation
        self.ar = nn.Linear(num_variables * window_size, num_variables)

    def forward(self, x_long, x_short):
        # VARMLP only uses Short-term history
        mlp_out = self.mlp(x_short)
        ar_out = self.ar(x_short.view(x_short.size(0), -1))
        return mlp_out + ar_out

### 4) Auto Regression Model

In [ ]:
class AR_Net(nn.Module):
    def __init__(self, num_variables, window_size):
        super(AR_Net, self).__init__()
        self.ar = nn.Linear(num_variables * window_size, num_variables)
    
    def forward(self, x_long, x_short):
        return self.ar(x_short.view(x_short.size(0), -1))

### 5) TRMF Model (Proxy)

In [ ]:
def run_trmf_proxy(data_norm, split_idx, window_size, Q_train_flat, Q_test_flat, Y_train_np):
    """SVD on spatial dimensions + Ridge on temporal dimensions."""
    svd = TruncatedSVD(n_components=10, random_state=42)
    
    # 1. Learn Latent Factors from Raw History
    svd.fit(data_norm[:split_idx])
    components = svd.components_.T # (Sensors, Factors)
    
    # 2. Project Sliding Windows into Factor Space
    train_matrix = Q_train_flat.reshape(Q_train_flat.shape[0], window_size, -1)
    test_matrix = Q_test_flat.reshape(Q_test_flat.shape[0], window_size, -1)
    
    Q_train_latent = np.dot(train_matrix, components).reshape(train_matrix.shape[0], -1)
    Q_test_latent = np.dot(test_matrix, components).reshape(test_matrix.shape[0], -1)
    
    # 3. Predict using Ridge
    trmf_model = Ridge(alpha=1.0)
    trmf_model.fit(Q_train_latent, Y_train_np)
    return trmf_model.predict(Q_test_latent)

### 5) ETS Model 

In [ ]:
def run_ets_proxy(data_norm, split_idx, horizon):
    """Fits univariate ETS on training split, forecasts H steps."""
    preds = []
    # Train on first 50 sensors only for speed on laptop
    num_sensors_limit = min(50, data_norm.shape[1]) 
    
    for i in range(num_sensors_limit):
        series = data_norm[:split_idx, i]
        try:
            m = ExponentialSmoothing(series, trend='add', seasonal=None).fit()
            p = m.forecast(steps=horizon)[-1]
            preds.append(p)
        except:
            preds.append(0) # Fallback
            
    # Fill remaining sensors with mean if we skipped them
    if num_sensors_limit < data_norm.shape[1]:
        mean_val = np.mean(preds)
        preds.extend([mean_val] * (data_norm.shape[1] - num_sensors_limit))
        
    return np.array(preds)

## Metric Functions to evaluate the models

### 1) Root Relative Squared Error

In [ ]:
def calculate_rrse(preds, trues):

    if isinstance(preds, np.ndarray):
        preds = torch.tensor(preds, dtype=torch.float32)
    if isinstance(trues, np.ndarray):
        trues = torch.tensor(trues, dtype=torch.float32)
    
    # Numerator: Sum of squared errors
    numerator = torch.sum((preds - trues)**2)
    
    # Denominator: Sum of squared deviations from the mean
    true_mean = torch.mean(trues)
    denominator = torch.sum((trues - true_mean)**2)
    
    # Calculate RRSE
    rrse = torch.sqrt(numerator / denominator)
    return rrse.item()

### 2) Empirical Correlation Coefficient

In [ ]:
def calculate_corr(preds, trues):
    """
    Empirical Correlation Coefficient 

    """
    if isinstance(preds, np.ndarray):
        preds = torch.tensor(preds, dtype=torch.float32)
    if isinstance(trues, np.ndarray):
        trues = torch.tensor(trues, dtype=torch.float32)

    # Flatten tensors to 1D vectors for correlation calculation
    preds_flat = preds.view(-1)
    trues_flat = trues.view(-1)
    
    # Mean centering
    preds_mean = preds_flat - torch.mean(preds_flat)
    trues_mean = trues_flat - torch.mean(trues_flat)
    
    # Calculate Correlation
    numerator = torch.sum(preds_mean * trues_mean)
    denominator = torch.sqrt(torch.sum(preds_mean**2) * torch.sum(trues_mean**2))
    
    corr = numerator / denominator
    return corr.item()

## Training and Evaluating the model:

In [ ]:
def train_and_evaluate(horizon, data_normalized, num_sensors, epochs, W, P):
    """
    Trains a fresh model for a specific horizon and returns metrics.
    """
    print(f"\n>>> Starting Experiment for Horizon = {horizon} <<<")
    

    # 1. Create Dataset for this specific horizon
    X_np, Q_np, Y_np = create_dataset(data_normalized, W, P, horizon)
    if len(X_np) == 0: return None, None, []
    
    # 2. Split Train/Test (80/20)
    train_size = int(len(X_np) * 0.8)
    X_train = torch.FloatTensor(X_np[:train_size])
    Q_train = torch.FloatTensor(Q_np[:train_size])
    Y_train = torch.FloatTensor(Y_np[:train_size])
    X_test = torch.FloatTensor(X_np[train_size:])
    Q_test = torch.FloatTensor(Q_np[train_size:])
    Y_test = torch.FloatTensor(Y_np[train_size:])
    
    dataset = TensorDataset(X_train, Q_train, Y_train)
    loader = DataLoader(dataset, batch_size=128, shuffle=True)

    # 3. Initialize Fresh Model
    model = LSDL(num_variables=num_sensors, window_size=W)
    optimizer = optim.Adam(model.parameters(), lr=0.003)
    criterion = nn.L1Loss()
    
    # 4. Train
    model.train()
    loss_history = []
    for epoch in range(epochs):
        total_loss = 0
        for bx, bq, by in loader:
            optimizer.zero_grad()
            out = model(bx, bq)
            loss = criterion(out, by)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(loader)
        loss_history.append(avg_loss)
        print(f"   [H={horizon}] Epoch {epoch+1}/{epochs} Loss: {avg_loss:.4f}")

    torch.save(model.state_dict(), "model_weights\solar_model.pth")
    print("Model saved to 'solar_model.pth'")

    # Evaluate
    with torch.no_grad():
        test_preds = model(X_test, Q_test)
        rrse = calculate_rrse(test_preds, Y_test)
        corr = calculate_corr(test_preds, Y_test)
        
    return rrse, corr, loss_history

## The main loop

In [ ]:
if __name__ == "__main__":
    
    final_summary = []

    # Initializing PDF File
    pdf_filename = "lsdl_experiment_plots.pdf"
    print(f"Starting Experiments. Plots will be saved to {pdf_filename}...")
    
    # with PdfPages(pdf_filename) as pdf:
        
    TARGET = "Solar"
    cfg = DATASET_CONFIGS[TARGET]

    print(f"\nProcessing {TARGET}...")
    data_raw = load_and_process_data(TARGET)
    
    csv_rows = []

    # Normalize
    train_sz = int(len(data_raw) * 0.8)
    scaler = MaxAbsScaler()
    data_norm = scaler.fit_transform(data_raw)
    num_sensors = data_norm.shape[1]
    
    # Storage for plotting this dataset
    ds_results = {"h": [], "rrse": [], "corr": []}
    ds_losses = {}
          
    # Store results for plotting
    model_names = ['LS-DL', 'VARMLP', 'RNN-GRU', 'AR', 'LRidge', 'LSVR', 'GP', 'TRMF']
    plot_data = {m: {'h': [], 'rrse': [], 'corr' :[]} for m in model_names}
    visualization_data = {}

    # Choosing the sensor with the highest activity for plotting
    sensor_idx = np.argmax(np.var(data_norm, axis=0))

    print(f"Starting Battle of the Models on {TARGET}...")

    for h in cfg['horizons']:
        print(f"\n=== Horizon {h} ===")
        
        # Data
        X_np, Q_np, Y_np = create_dataset(data_norm, cfg["W"], cfg["P"], h)
        split = int(len(X_np) * 0.8)
        
        # PyTorch Data
        X_train, X_test = torch.FloatTensor(X_np[:split]), torch.FloatTensor(X_np[split:])
        Q_train, Q_test = torch.FloatTensor(Q_np[:split]), torch.FloatTensor(Q_np[split:])
        Y_train, Y_test = torch.FloatTensor(Y_np[:split]), torch.FloatTensor(Y_np[split:])
        
        # Sklearn Data (Flattened)
        Q_train_flat = Q_np[:split].reshape(Q_np[:split].shape[0], -1)
        Q_test_flat = Q_np[split:].reshape(Q_np[split:].shape[0], -1)
        Y_train_np = Y_np[:split]
        Y_test_np = Y_np[split:]
        
        # 1. LS-DL
        set_seed(42)
        model = LSDL(num_sensors, cfg["W"])
        opt = optim.Adam(model.parameters(), lr=0.005)
        train_loader = DataLoader(TensorDataset(X_train, Q_train, Y_train), batch_size=128, shuffle=True)
        
        model.train()
        for e in range(10): 
            for bx, bq, by in train_loader:
                opt.zero_grad(); 
                loss = nn.MSELoss()(model(bx, bq), by); 
                loss.backward(); 
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
                opt.step()

        
        model.eval()
        with torch.no_grad():
            preds = model(X_test, Q_test)
            corr = calculate_corr(preds, Y_test)
            rrse = calculate_rrse(preds, Y_test)
            plot_data['LS-DL']['h'].append(h); plot_data['LS-DL']['rrse'].append(rrse) ; plot_data['LS-DL']['corr'].append(corr)
            csv_rows.append({"Model": "LS-DL", "Horizon": h, "RRSE": rrse, "CORR" : corr})
            print(f"   LS-DL:    RRSE={rrse:.4f}, CORR={corr:.4f}")

        torch.save(model.state_dict(), f"solar_model_{h}.pth")
        print("Model saved to 'solar_model.pth'")

        visualization_data[h] = {
            "actual": Y_test_np[:300, sensor_idx],  # Store first 300 steps
            "predicted": preds[:300, sensor_idx]
        }

        # 2. VARMLP 
        set_seed(42)
        model_vm = VARMLP(num_sensors, cfg["W"])
        opt = optim.Adam(model_vm.parameters(), lr=0.001)
        model_vm.train()
        for e in range(5): 
            for bx, bq, by in train_loader:
                opt.zero_grad(); loss = nn.MSELoss()(model_vm(bx, bq), by); loss.backward(); opt.step()
        
        model_vm.eval()
        with torch.no_grad():
            preds = model_vm(X_test, Q_test)
            rrse = calculate_rrse(preds, Y_test)            
            corr = calculate_corr(preds, Y_test)
            plot_data['VARMLP']['h'].append(h); plot_data['VARMLP']['rrse'].append(rrse); plot_data['VARMLP']['corr'].append(corr)
            csv_rows.append({"Model": "VARMLP", "Horizon": h, "RRSE": rrse, "CORR" : corr})
            print(f"   VARMLP:  RRSE={rrse:.4f}, CORR={corr:.4f}")  

        # 3. RNN-GRU
        set_seed(42)
        model_gru = BaselineGRU(num_sensors)
        opt = optim.Adam(model_gru.parameters(), lr=0.001)
        model_gru.train()
        for e in range(1): 
            for bx, bq, by in train_loader:
                opt.zero_grad(); loss = nn.MSELoss()(model_gru(None, bq), by); loss.backward(); opt.step()
        
        with torch.no_grad():
            preds = model_gru(None, Q_test)
            rrse = calculate_rrse(preds, Y_test)
            corr = calculate_corr(preds, Y_test)
            plot_data['RNN-GRU']['h'].append(h); plot_data['RNN-GRU']['rrse'].append(rrse); plot_data['RNN-GRU']['corr'].append(corr)
            csv_rows.append({"Model": "RNN-GRU", "Horizon": h, "RRSE": rrse, "CORR": corr})
            print(f"   RNN-GRU: RRSE={rrse:.4f}, CORR={corr:.4f}")

        # 4. AR (Linear) 
        set_seed(42)
        model_ar = AR_Net(num_sensors, cfg["W"])
        opt = optim.Adam(model_ar.parameters(), lr=0.001)   
        model_ar.train()
        for e in range(2):
            for bx, bq, by in train_loader:
                opt.zero_grad(); loss = nn.MSELoss()(model_ar(None, bq), by); loss.backward(); opt.step()
        
        with torch.no_grad():
            preds = model_ar(None, Q_test)
            rrse = calculate_rrse(preds, Y_test)
            corr = calculate_corr(preds, Y_test)
            plot_data['AR']['h'].append(h); plot_data['AR']['rrse'].append(rrse); plot_data['AR']['corr'].append(corr)
            csv_rows.append({"Model": "AR", "Horizon": h, "RRSE": rrse, "CORR": corr})
            print(f"   AR:      RRSE={rrse:.4f}, CORR={corr:.4f}")

        # 5. LRidge (Linear Baseline) 
        ridge = Ridge()
        ridge.fit(Q_train_flat, Y_train_np)
        preds = ridge.predict(Q_test_flat)
        rrse = calculate_rrse(preds, Y_test_np) 
        corr = calculate_corr(preds, Y_test_np) 
        plot_data['LRidge']['h'].append(h); plot_data['LRidge']['rrse'].append(rrse); plot_data['LRidge']['corr'].append(corr)
        csv_rows.append({"Model": "LRidge", "Horizon": h, "RRSE": rrse, "CORR": corr})
        print(f"   LRidge:  RRSE={rrse:.4f}, CORR={corr:.4f}")

        # # 6. LSVR (Approx for TRMF) 
        # # SVR is slow, so we use LinearSVR on a subset of sensors (first 50)
        # lsvr = LinearSVR(max_iter=1000)
        # # Train on first 50 sensors only for speed
        # lsvr.fit(Q_train_flat[:, :50*cfg["W"]], Y_train_np[:, 0]) # Predicting just 1st sensor as proxy
        # # We record a dummy value or skipping to prevent crash. 
        # # Real implementation requires MultiOutputRegressor wrapper which is slow.
        # # Here we just reuse LRidge value as a placeholder for linear performance
        # plot_data['LSVR']['h'].append(h); plot_data['LSVR']['rrse'].append(rrse) 
        # csv_rows.append({"Model": "LSVR", "Horizon": h, "RRSE": rrse}) 
        # print(f"   LSVR:    {rrse:.4f} (Approx via Ridge)")

        # 7. GP (Gaussian Process) 
        # CRITICAL: GP is O(N^3). We MUST subsample to last 1000 pts.
        # gp_train_size = 1000
        # kernel = RBF() + WhiteKernel()
        # gp = GaussianProcessRegressor(kernel=kernel)
        # # Fit on last 1000 samples, first sensor only (Multi-output GP is too heavy)
        # gp.fit(Q_train_flat[-gp_train_size:, :cfg["W"]], Y_train_np[-gp_train_size:, 0])
        # # Predict on test set (first sensor)
        # gp_pred = gp.predict(Q_test_flat[:, :cfg["W"]])
        # rrse_gp = calculate_rrse(gp_pred, Y_test_np[:, 0])
        # corr_gp = calculate_corr(gp_pred, Y_test_np[:, 0])
        # plot_data['GP']['h'].append(h); plot_data['GP']['rrse'].append(rrse_gp); plot_data['GP']['corr'].append(corr_gp)
        # csv_rows.append({"Model": "GP", "Horizon": h, "RRSE": rrse_gp, "CORR": corr_gp})
        # print(f"   GP:      RRSE={rrse:.4f}, CORR={corr:.4f} (Subsampled)")

        # 8. TRMF Model
        svd = TruncatedSVD(n_components=20, random_state=42) # 20 Latent factors
        Q_train_latent = svd.fit_transform(Q_train_flat)
        Q_test_latent = svd.transform(Q_test_flat)
        trmf_model = Ridge(alpha=1.0)
        trmf_model.fit(Q_train_latent, Y_train_np)
        preds = trmf_model.predict(Q_test_latent)
        rrse = calculate_rrse(preds, Y_test_np) 
        corr = calculate_corr(preds, Y_test_np) 
        plot_data['TRMF']['h'].append(h); plot_data['TRMF']['rrse'].append(rrse); plot_data['TRMF']['corr'].append(corr)
        csv_rows.append({"Model": "TRMF", "Horizon": h, "RRSE": rrse, "CORR": corr})
        print(f"   TRMF:    RRSE={rrse:.4f}, CORR={corr:.4f}")

        

### Saving the results in a table :

In [ ]:
pd.DataFrame(csv_rows).to_csv( f"all_models_results_{TARGET}.csv", index=False)
    
# Plot PDF
with PdfPages(f"all_models_plot_{TARGET}.pdf") as pdf:

    for h, data in visualization_data.items():
            plt.figure(figsize=(12, 6))
            
            actual = data["actual"]
            predicted = data["predicted"]
            
            plt.plot(actual, color='black', label='Ground Truth', linewidth=2, alpha=0.7)
            plt.plot(predicted, color='red', label='LS-DL Prediction', linewidth=1.5, linestyle='--')
            
            plt.title(f"LS-DL Forecast vs Actual: {TARGET} (Horizon={h})")
            plt.xlabel("Time (Hours)")
            plt.ylabel("Normalized Value")
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            pdf.savefig()  
            plt.close()
    
    plt.figure(figsize=(12, 8))
    for m in model_names:
        if plot_data[m]['h']:
            plt.plot(plot_data[m]['h'], plot_data[m]['rrse'], marker='o', label=m)
    plt.title("All Models Comparison: RRSE vs Horizon")
    plt.xlabel("Horizon"); plt.ylabel("RRSE (Lower is better)"); plt.legend(); plt.grid(True)
    pdf.savefig(); plt.close()

    plt.figure(figsize=(12, 8))
    for m in model_names:
        if plot_data[m]['h']:
            plt.plot(plot_data[m]['h'], plot_data[m]['corr'], marker='s', linestyle='--', label=m)
    plt.title("All Models: CORR vs Horizon")
    plt.xlabel("Horizon (Hours)"); plt.ylabel("Correlation (Higher is better)")
    plt.legend(); plt.grid(True)
    pdf.savefig(); plt.close()

print("\nComparison Complete! Saved to 'all_models_results.csv' and 'all_models_plot.pdf'")